# Homework 3 Solutions

1. Download daily price data for QQQ, TLT, GLD, RWO from yahoo finance since 2016-01-01. Using the adjusted close price data, compute daily returns. This should be a DataFrame with index=date, columns=ticker and values=daily returns.

In [1]:
import yfinance as yf

In [2]:
univ = ['QQQ','TLT','GLD','RWO']
px = yf.download(univ, start="2024-01-01")

[*********************100%%**********************]  4 of 4 completed


In [3]:
adj_close = px['Adj Close']
ret = adj_close / adj_close.shift() - 1

2. Compute the signal. Complete the compute_momentum function which computes a simple momentum signal. The function takes in a DataFrame with index = date, columns=ticker and values containing daily returns. It returns a new DataFrame with index = date, columns=ticker and values containing the momentum signal for the ticker on that day. The momentum signal for each ticker is defined as the annualized sharpe ratio of the past 252 days.

In [4]:
import math

def compute_momentum(ret):
    momentum = ret.rolling(252).mean()/ret.rolling(252).std()*math.sqrt(252)
    return momentum

momentum = compute_momentum(ret)
momentum

Ticker,GLD,QQQ,RWO,TLT
Date,,,,
2024-01-02,NaN,NaN,NaN,NaN
2024-01-03,NaN,NaN,NaN,NaN
2024-01-04,NaN,NaN,NaN,NaN
2024-01-05,NaN,NaN,NaN,NaN
2024-01-08,NaN,NaN,NaN,NaN
...,...,...,...,...
2024-07-16,NaN,NaN,NaN,NaN
2024-07-17,NaN,NaN,NaN,NaN
2024-07-18,NaN,NaN,NaN,NaN


3. Create a portfolio. Complete the function compute_portfolio. This function takes as input the DataFrame "momentum" from above. It returns a new DataFrame "portfolio" which has the same index/columns and has as values portfolio weights. The weights are computed as follows. On each date, equal-weight long the tickers with a momentum signal above 1.

In [5]:
def compute_portfolio(momentum):
    portfolio = (momentum > 1)*1
    portfolio = portfolio.div(portfolio.abs().sum(1),0)
    return portfolio

portfolio = compute_portfolio(momentum)

4. Portfolio returns. Using the "portfolio" returned in part(3) and the returns generated in part(1), compute the returns to the simple momentum strategy. 
- What is the annualized sharpe ratio of the strategy? 
- How about the annualized sharpe ratio within each year? 
- How correlated is the strategy with the underlying tickers?
- Plot the cumulative sum of the returns through time

In [6]:
strat_ret = (portfolio.shift()*ret).sum(1)
# minor, but should start returns when signal starts
strat_ret = strat_ret.loc[momentum.dropna().index[0]:] 

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# sharpe
strat_ret.mean()/strat_ret.std()*math.sqrt(252)

In [ ]:
# sharpe within each year
import numpy as np 

sharpe = lambda x: x.mean()/x.std()*math.sqrt(252) 
strat_ret.groupby([x.year for x in strat_ret.index]).apply(sharpe)

In [ ]:
# correlation
ret.corrwith(strat_ret)

In [ ]:
# cumulative returns
strat_ret.cumsum().plot()